
Imports e definição do dataset.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras

keras.utils.set_random_seed(812)

Carregamento do dataset cifar-10

In [ ]:
# Carregar o conjunto de dados CIFAR-10
cifar10 = keras.datasets.cifar10
#Carrega duas tuplas, representando os dados de treinamento e de teste.
#Cada tupla tem as imagens e os respectivos rótulos
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
num_classes = 10

O código abaixo mostra as 10 primeiras imagens de treino e teste do cifar-10

In [ ]:
# Defina as classes do CIFAR-10
class_names = ['Avião', 'Automóvel', 'Pássaro', 'Gato', 'Cervo',
               'Cachorro', 'Sapo', 'Cavalo', 'Navio', 'Caminhão']

# Crie um dicionário para mapear as classes para as imagens correspondentes
class_to_image = {}
for i in range(10):
    index = (test_labels == i).nonzero()[0][0]  # Encontre o primeiro índice da classe
    class_to_image[i] = test_images[index]

# Mostre uma imagem de cada classe
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.xticks([])  # Remova os rótulos do eixo x
    plt.yticks([])  # Remova os rótulos do eixo y
    plt.imshow(class_to_image[i])
    plt.xlabel(class_names[i])

plt.tight_layout()
plt.show()

Abaixo, convertemos os rótulos escalares (números de 0 a 9) para one-hot encoding.

Não é necessário realizar este passo, caso seja utilizada a função de custo esparse_categorical_cross_entropy

In [ ]:
# Converter para codificação one-hot dos labels
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)
#Não é necessário se utilizar como função de custo esparse_categorical_cross_entropy

Função que retorna uma rede neural para o cifar-10

In [ ]:
# Crie o modelo de rede neural convolucional simples
def get_cifar10_network():
    model = keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),#(32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')  # 10 classes de saída
    ])

    # Compile o modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                  metrics=['accuracy'])

    model.summary()

    return model

Trecho para treinar e avaliar a rede neural.
O treino é realizado com os dados de treino e a avaliação do modelo é realizada nos dados de teste.

In [ ]:
# Treine o modelo
model = get_cifar10_network()
model.fit(train_images, train_labels, epochs=10)

# Avalie o modelo no conjunto de teste
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Acurácia no conjunto de teste: {test_accuracy * 100:.2f}%')

Na célula abaixo, adicione o código para carregar os demais datasets

In [ ]:
# Inclua o código para carregar os demais datasets
import numpy as np
import numpy.typing as npt
import typing

from dataclasses import dataclass


@dataclass
class Dataset:
    train_x: npt.NDArray[np.floating]
    train_y: npt.NDArray[np.integer]

    test_x: npt.NDArray[np.floating]
    test_y: npt.NDArray[np.integer]

    n_classes: int

    @classmethod
    def from_keras(
        cls,
        keras_dataset: tuple[
            tuple[npt.NDArray, npt.NDArray], tuple[npt.NDArray, npt.NDArray]
        ],
        *,
        n_classes: int,
    ) -> typing.Self:
        return cls(*keras_dataset[0], *keras_dataset[1], n_classes=n_classes)
    
    def to_categorical_labels(self) -> None:
        self.train_y = tf.keras.utils.to_categorical(self.train_y, num_classes=self.n_classes)
        self.test_y = tf.keras.utils.to_categorical(self.test_y, num_classes=self.n_classes)


# mninst = keras.datasets.mnist
# (train_images, train_labels), (test_images, test_labels) = mninst.load_data()
# fashion_mnist = keras.datasets.fashion_mnist
# (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# cifar100 = keras.datasets.cifar100
# (train_images, train_labels), (test_images, test_labels) = cifar100.load_data()
# num_classes = 100

cifar10 = Dataset.from_keras(keras.datasets.cifar10.load_data(), n_classes=10)
cifar100 = Dataset.from_keras(
    keras.datasets.cifar100.load_data(label_mode="fine"), n_classes=100
)
mnist = Dataset.from_keras(keras.datasets.mnist.load_data(), n_classes=10)
fashion_mnist = Dataset.from_keras(
    keras.datasets.fashion_mnist.load_data(), n_classes=10
)

Tarefa
Escreva código para executar redes neurais nos seguintes datasets:

MNIST (pode aproveitar o codigo existente)
Fashion MNIST
CIFAR-10
CIFAR-100
Cada execução deve ser por 10 épocas.

Você deve preencher as funções a seguir para retornarem a rede neural com a melhor configuração que você conseguiu para cada dataset. O notebook deve ser entregue com a rede neural que obteve a melhor performance em cada conjunto de dados.

IMPORTANTE: as funções não devem TREINAR nem AVALIAR as redes neurais, apenas instanciá-las e retorná-las.

Ao final, preencha o dict results com o desempenho encontrado em cada execução.

In [ ]:
def get_fashion_mnist_network():
    num_classes = 10
    model = tf.keras.models.Sequential(
        name="fashion_mnist",
        layers=[
            tf.keras.layers.Conv2D(
                16, (3, 3), activation="relu", input_shape=(28, 28, 1)
            ),  # 1 camada de convolução: 16 filtros 3x3, stride padrão (1,1), ativação relu
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool2D(
                (2, 2),
            ),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

    model.summary()

    return model


def get_mnist_network():
    num_classes = 10
    model = tf.keras.models.Sequential(
        name="mnist",
        layers=[
            tf.keras.layers.Conv2D(
                16, (3, 3), activation="relu", input_shape=(28, 28, 1)
            ),  # 1 camada de convolução: 16 filtros 3x3, stride padrão (1,1), ativação relu
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
            tf.keras.layers.MaxPool2D(
                (2, 2),
            ),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

    model.summary()

    return model


def get_cifar100_network():
    num_classes = 100
    model = keras.Sequential(
        name="cifar100",
        layers=[
            tf.keras.layers.Conv2D(
                32, (3, 3), activation="relu", input_shape=(32, 32, 3)
            ),  # (32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(
                num_classes, activation="softmax"
            ),  # 100 classes de saída
        ]
    )

    # Compile o modelo
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    model.summary()

    return model


# Você pode aproveitar as ideias do exemplo acima
def get_cifar10_network():
    num_classes = 10
    model = keras.Sequential(
        name="cifar10",
        layers=[
            tf.keras.layers.Conv2D(
                16, (3, 3), activation="relu", input_shape=(32, 32, 3)
            ),  # (32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Dropout(0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(
                num_classes, activation="softmax"
            ),  # 10 classes de saída
        ]
    )

    # Compile o modelo
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    model.summary()

    return model

In [ ]:
# Converter para codificação one-hot dos labels

# train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)
# test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)

# Não é necessário se utilizar como função de custo esparse_categorical_cross_entropy

cifar10.to_categorical_labels()
cifar100.to_categorical_labels()
mnist.to_categorical_labels()
fashion_mnist.to_categorical_labels()

In [ ]:
cifar10_model = get_cifar10_network()
cifar10_model.fit(cifar10.train_x, cifar10.train_y, epochs=10)

cifar100_model = get_cifar100_network()
cifar100_model.fit(cifar100.train_x, cifar100.train_y, epochs=10)

mnist_model = get_mnist_network()
mnist_model.fit(mnist.train_x, mnist.train_y, epochs=10)

fashion_mnist_model = get_fashion_mnist_network()
fashion_mnist_model.fit(fashion_mnist.train_x, fashion_mnist.train_y, epochs=10)

In [ ]:
_datasets = (cifar10, cifar100, mnist, fashion_mnist)
_models = (cifar10_model, cifar100_model, mnist_model, fashion_mnist_model)

for dataset, model in zip(_datasets, _models):
    test_loss, test_accuracy = model.evaluate(dataset.test_x, dataset.test_y)
    print(f"Acurácia no conjunto de teste: {test_accuracy * 100:.2f}%")

Preencha o dict abaixo substituindo os None com a acuracia final (acc) e o tempo de treinamento (time) encontrado no seu experimento pra cada dataset

In [ ]:
results = {
    "mnist": {"time": 441, "acc": 0.9932},
    "fashion_mnist": {"time": 692, "acc": 0.9612},
    "cifar10": {"time": 579, "acc": 0.7977},
    "cifar100": {"time": 1278, "acc": 0.4246},
}